In [52]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [53]:
# Drop unneeded columns
raw_df = pd.read_csv("../res/responses.csv")
raw_df.drop(["Timestamp", "Name (Optional)"], axis=1, inplace=True);

In [54]:
# Change column names
cols = raw_df.columns
new_cols = []
for col in cols:
    try:
        new_cols.append(col[col.index("[")+1:-1])
    except ValueError:
        new_cols.append(col)
raw_df.columns = new_cols

to_replace = {
    "Would you agree that you frequently shop online?":"Frequency",
    "Where do you shop more often?":"Where",
    "BACK TO SCHOOL - Please select all items that you would typically buy during this season.":"Back To School",
    "SUMMER - Please select all items that you would typically buy during this season.":"Summer",
    "Home Entertainment (TV, Speakers, Online Streaming Subscription, Etc.)":"Home Entertainment",
    "Current Region ":"Current Region"
}
raw_df.rename(columns = to_replace, inplace = True)

raw_df.columns

Index(['Age', 'Gender', 'Marital Status', 'Employment Status',
       'Current Region', 'Frequency', 'Where', 'Apparel/Clothing',
       'Health & Personal Care', 'Home Entertainment', 'Babies & Kids',
       'Food/Groceries', 'Toys, Games, Collectibles', 'Bags', 'Stationery',
       'Books', 'Sports & Travel', 'Wearable Accessories',
       'Automobile Accessories', 'Cameras/Camera Accessories',
       'Laptops/Computers', 'Tech Accessories', 'Pet Care/Accessories',
       'Power Tools', 'Back To School', 'Summer'],
      dtype='object')

In [55]:
# Create baskets according to season

user_data_cols = ['Age', 'Gender', 'Marital Status', 'Employment Status', 'Current Region', 'Frequency', 'Where']

# Create baskets for xmas, new year, when needed
holidays_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
bdays_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
periods_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
wneeded_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
seasonal_baskets = [holidays_basket, bdays_basket, periods_basket, wneeded_basket]

# Change values accordingly
keywords = ["Christmas", "Birthdays", "Monthly", "Needed"]
i = 0
for basket in seasonal_baskets:
    cols = basket.columns
    keyword = keywords[i]
    for col in cols:
        if col not in user_data_cols:
            basket[col] = np.where(~(basket[col].str.find(keyword) > -1), 0, 1)
    i += 1 

wneeded_basket.head()

,Age,Gender,Marital Status,Employment Status,Current Region,Frequency,Where,Apparel/Clothing,Health & Personal Care,Home Entertainment,...,Stationery,Books,Sports & Travel,Wearable Accessories,Automobile Accessories,Cameras/Camera Accessories,Laptops/Computers,Tech Accessories,Pet Care/Accessories,Power Tools
0,21,Male,Single,Employed,Region III - Central Luzon,2,Online Stores,1,1,1,...,0,0,0,1,0,0,0,0,0,0
1,21,Male,Single,Student,CAR - Cordillera Administrative Region,1,Online Stores,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2,20,Male,Single,Student,Region III - Central Luzon,2,Online Stores,1,0,0,...,1,1,1,1,1,1,1,0,1,1
3,21,Female,Single,Student,Region IV-A - CALABARZON,4,Online Stores,0,0,1,...,1,1,1,0,0,0,1,1,1,0
4,22,Female,Single,Student,Region IV-A - CALABARZON,5,Online Stores,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# Possible answers of respondents in survey (Also the items available for each season)
summer_items = ["Clothing", "Umbrellas", "Sunscreen/Heat-Protectant Skin Products", 
"Swimsuit", "Swim Cap", "Swim Goggles", "Swim Snorkles", "Towels", "Bags", "Water Bottles", 
"Ice Cooler", "Inflatable Pools", "Pool Toys", "Swimming Safety Equipment (Swimming Rings/Life Jacket/Floaties/Etc.)", "Electric Fan"]

bts_items = ["Pen/Marker/Pencil/Scriber", "Paper/Envelopes/Post-it Notes/", "Notebooks", 
"Academic Books", "Art Materials", "Laptop", "Phone/Tablet", "Tech Products/Tech Accessories", 
"School Uniforms/Clothing", "Umbrella", "Raincoat", "Bags/School Bags", "Make up/Beauty Products"]

In [57]:
# Create baskets according to season part 2
summer_basket = raw_df[user_data_cols]
bts_basket = raw_df[user_data_cols]

# Set appropriate columns
summer_basket = summer_basket.reindex(columns = summer_basket.columns.tolist() + summer_items)
bts_basket = bts_basket.reindex(columns = bts_basket.columns.tolist() + bts_items)

# Assign appropriate values in summer basket
for col in summer_basket.columns:
    if col not in user_data_cols:
        summer_basket[col] = np.where(~(raw_df["Summer"].str.find(col) > -1), 0, 1)
# Assign Appropriate Values in Back To School basket
for col in bts_basket.columns:
    if col not in user_data_cols:
        bts_basket[col] = np.where(~(raw_df["Back To School"].str.find(col) > -1), 0, 1)

# Clean column name
to_replace_summer = {
    "Swimming Safety Equipment (Swimming Rings/Life Jacket/Floaties/Etc.)":"Swimming Safety Equipment",
}
summer_basket.rename(columns = to_replace_summer, inplace = True)

summer_basket.head()

,Age,Gender,Marital Status,Employment Status,Current Region,Frequency,Where,Clothing,Umbrellas,Sunscreen/Heat-Protectant Skin Products,...,Swim Goggles,Swim Snorkles,Towels,Bags,Water Bottles,Ice Cooler,Inflatable Pools,Pool Toys,Swimming Safety Equipment,Electric Fan
0,21,Male,Single,Employed,Region III - Central Luzon,2,Online Stores,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,21,Male,Single,Student,CAR - Cordillera Administrative Region,1,Online Stores,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,20,Male,Single,Student,Region III - Central Luzon,2,Online Stores,1,1,1,...,1,0,0,0,1,0,0,0,0,0
3,21,Female,Single,Student,Region IV-A - CALABARZON,4,Online Stores,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,22,Female,Single,Student,Region IV-A - CALABARZON,5,Online Stores,1,0,1,...,0,0,0,0,0,0,0,0,0,0


List of baskets created:

- holidays_basket (xmas, new year)

- bdays_basket (self/others)

- periods_basket (monthly/weekly/shopping sales)

- wneeded_basket (when needed)

- summer_basket (summer)

- bts_basket (back to school)

In [58]:
# Create a dataframe of all baskets
basket_names = ["holidays_basket", "bdays_basket", "periods_basket", "wneeded_basket", "summer_basket", "bts_basket"]
all_baskets = [holidays_basket, bdays_basket, periods_basket, wneeded_basket, summer_basket, bts_basket]
# initialize minimum supports for each basket here
min_supports = [0.05, 0.1, 0.05, 0.5, 0.1, 0.2]
baskets = pd.DataFrame({"name":basket_names,"basket":all_baskets, "min_support":min_supports})
baskets

,name,basket,min_support
0,holidays_basket,Age Gender Marital Status Employment Stat...,0.05
1,bdays_basket,Age Gender Marital Status Employment Stat...,0.10
2,periods_basket,Age Gender Marital Status Employment Stat...,0.05
3,wneeded_basket,Age Gender Marital Status Employment Stat...,0.50
4,summer_basket,Age Gender Marital Status Employment Stat...,0.10
5,bts_basket,Age Gender Marital Status Employment Stat...,0.20


In [59]:
# Get rules from all generalized baskets:

for i in baskets.index:
    # Indicate which basket to show rules:
    row = baskets.iloc[i]
    basket_name = row["name"]
    basket = row["basket"]
    min_support = row["min_support"]
    print(f'basket name: {basket_name}')

    # Sort the holiday basket
    # Sort the basket before dropping user_data_cols
    # Example: sort it such that only female respondents are left
    basket_sorted = basket.drop(user_data_cols, axis=1)

    # Get frequent item set
    freq_items = apriori(basket_sorted, min_support = min_support, use_colnames=True)
    freq_items.sort_values("support", ascending=False)

    print(f'length of frequent item sets: {len(freq_items)}')
    print(f'minimum support: {min_support}')
    # Get rules 
    rules = association_rules(freq_items, metric="lift", min_threshold=1)
    display(rules.sort_values("support", ascending=False).head(10))

basket name: holidays_basket
length of frequent item sets: 35
minimum support: 0.05


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Wearable Accessories),(Apparel/Clothing),0.208791,0.329670,0.120879,0.578947,1.756140,0.052047,1.592033
14,(Apparel/Clothing),(Wearable Accessories),0.329670,0.208791,0.120879,0.366667,1.756140,0.052047,1.249277
12,(Apparel/Clothing),(Sports & Travel),0.329670,0.120879,0.087912,0.266667,2.206061,0.048062,1.198801
36,(Wearable Accessories),(Bags),0.208791,0.131868,0.087912,0.421053,3.192982,0.060379,1.499500
37,(Bags),(Wearable Accessories),0.131868,0.208791,0.087912,0.666667,3.192982,0.060379,2.373626
13,(Sports & Travel),(Apparel/Clothing),0.120879,0.329670,0.087912,0.727273,2.206061,0.048062,2.457875
8,(Apparel/Clothing),(Bags),0.329670,0.131868,0.087912,0.266667,2.022222,0.044439,1.183816
9,(Bags),(Apparel/Clothing),0.131868,0.329670,0.087912,0.666667,2.022222,0.044439,2.010989
11,(Books),(Apparel/Clothing),0.087912,0.329670,0.076923,0.875000,2.654167,0.047941,5.362637
42,"(Apparel/Clothing, Wearable Accessories)",(Bags),0.120879,0.131868,0.076923,0.636364,4.825758,0.060983,2.387363


basket name: bdays_basket
length of frequent item sets: 32
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(Apparel/Clothing),(Wearable Accessories),0.318681,0.340659,0.219780,0.689655,2.024472,0.111218,2.124542
7,(Wearable Accessories),(Apparel/Clothing),0.340659,0.318681,0.219780,0.645161,2.024472,0.111218,1.920080
15,"(Toys, Games, Collectibles)",(Wearable Accessories),0.373626,0.340659,0.186813,0.500000,1.467742,0.059534,1.318681
14,(Wearable Accessories),"(Toys, Games, Collectibles)",0.340659,0.373626,0.186813,0.548387,1.467742,0.059534,1.386970
19,(Tech Accessories),"(Toys, Games, Collectibles)",0.274725,0.373626,0.175824,0.640000,1.712941,0.073180,1.739927
18,"(Toys, Games, Collectibles)",(Tech Accessories),0.373626,0.274725,0.175824,0.470588,1.712941,0.073180,1.369963
21,(Bags),(Wearable Accessories),0.241758,0.340659,0.153846,0.636364,1.868035,0.071489,1.813187
20,(Wearable Accessories),(Bags),0.340659,0.241758,0.153846,0.451613,1.868035,0.071489,1.382676
35,(Tech Accessories),(Laptops/Computers),0.274725,0.208791,0.131868,0.480000,2.298947,0.074508,1.521555
34,(Laptops/Computers),(Tech Accessories),0.208791,0.274725,0.131868,0.631579,2.298947,0.074508,1.968603


basket name: periods_basket
length of frequent item sets: 34
minimum support: 0.05


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
26,(Laptops/Computers),(Tech Accessories),0.747253,0.681319,0.582418,0.779412,1.143975,0.073300,1.444689
27,(Tech Accessories),(Laptops/Computers),0.681319,0.747253,0.582418,0.854839,1.143975,0.073300,1.741148
13,(Stationery),(Laptops/Computers),0.692308,0.747253,0.582418,0.841270,1.125817,0.065089,1.592308
12,(Laptops/Computers),(Stationery),0.747253,0.692308,0.582418,0.779412,1.125817,0.065089,1.394872
20,(Laptops/Computers),(Sports & Travel),0.747253,0.604396,0.549451,0.735294,1.216578,0.097814,1.494505
21,(Sports & Travel),(Laptops/Computers),0.604396,0.747253,0.549451,0.909091,1.216578,0.097814,2.780220
19,(Books),(Laptops/Computers),0.626374,0.747253,0.538462,0.859649,1.150413,0.070402,1.800824
18,(Laptops/Computers),(Books),0.747253,0.626374,0.538462,0.720588,1.150413,0.070402,1.337189
28,(Laptops/Computers),(Pet Care/Accessories),0.747253,0.626374,0.538462,0.720588,1.150413,0.070402,1.337189
29,(Pet Care/Accessories),(Laptops/Computers),0.626374,0.747253,0.538462,0.859649,1.150413,0.070402,1.800824


basket name: summer_basket
length of frequent item sets: 90
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Clothing),(Sunscreen/Heat-Protectant Skin Products),0.736264,0.560440,0.417582,0.567164,1.011999,0.004951,1.015536
1,(Sunscreen/Heat-Protectant Skin Products),(Clothing),0.560440,0.736264,0.417582,0.745098,1.011999,0.004951,1.034658
27,(Sunscreen/Heat-Protectant Skin Products),(Water Bottles),0.560440,0.538462,0.395604,0.705882,1.310924,0.093829,1.569231
26,(Water Bottles),(Sunscreen/Heat-Protectant Skin Products),0.538462,0.560440,0.395604,0.734694,1.310924,0.093829,1.656805
4,(Clothing),(Towels),0.736264,0.406593,0.329670,0.447761,1.101251,0.030310,1.074547
5,(Towels),(Clothing),0.406593,0.736264,0.329670,0.810811,1.101251,0.030310,1.394035
37,(Water Bottles),(Towels),0.538462,0.406593,0.318681,0.591837,1.455598,0.099746,1.453846
36,(Towels),(Water Bottles),0.406593,0.538462,0.318681,0.783784,1.455598,0.099746,2.134615
22,(Towels),(Sunscreen/Heat-Protectant Skin Products),0.406593,0.560440,0.307692,0.756757,1.350291,0.079821,1.807082
23,(Sunscreen/Heat-Protectant Skin Products),(Towels),0.560440,0.406593,0.307692,0.549020,1.350291,0.079821,1.315815


basket name: bts_basket
length of frequent item sets: 248
minimum support: 0.2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Pen/Marker/Pencil/Scriber),(Notebooks),0.956044,0.879121,0.846154,0.885057,1.006753,0.005676,1.051648
3,(Notebooks),(Pen/Marker/Pencil/Scriber),0.879121,0.956044,0.846154,0.962500,1.006753,0.005676,1.172161
0,(Pen/Marker/Pencil/Scriber),(Paper/Envelopes/Post-it Notes/),0.956044,0.813187,0.791209,0.827586,1.017707,0.013766,1.083516
1,(Paper/Envelopes/Post-it Notes/),(Pen/Marker/Pencil/Scriber),0.813187,0.956044,0.791209,0.972973,1.017707,0.013766,1.626374
78,"(Pen/Marker/Pencil/Scriber, Notebooks)",(Paper/Envelopes/Post-it Notes/),0.846154,0.813187,0.692308,0.818182,1.006143,0.004227,1.027473
80,(Pen/Marker/Pencil/Scriber),"(Notebooks, Paper/Envelopes/Post-it Notes/)",0.956044,0.714286,0.692308,0.724138,1.013793,0.009419,1.035714
81,(Paper/Envelopes/Post-it Notes/),"(Pen/Marker/Pencil/Scriber, Notebooks)",0.813187,0.846154,0.692308,0.851351,1.006143,0.004227,1.034965
79,"(Notebooks, Paper/Envelopes/Post-it Notes/)",(Pen/Marker/Pencil/Scriber),0.714286,0.956044,0.692308,0.969231,1.013793,0.009419,1.428571
36,(Notebooks),(Bags/School Bags),0.879121,0.637363,0.593407,0.675000,1.059052,0.033088,1.115807
37,(Bags/School Bags),(Notebooks),0.637363,0.879121,0.593407,0.931034,1.059052,0.033088,1.752747


In [60]:
basket.head()

,Age,Gender,Marital Status,Employment Status,Current Region,Frequency,Where,Pen/Marker/Pencil/Scriber,Paper/Envelopes/Post-it Notes/,Notebooks,Academic Books,Art Materials,Laptop,Phone/Tablet,Tech Products/Tech Accessories,School Uniforms/Clothing,Umbrella,Raincoat,Bags/School Bags,Make up/Beauty Products
0,21,Male,Single,Employed,Region III - Central Luzon,2,Online Stores,1,1,1,0,0,0,0,0,0,1,0,1,0
1,21,Male,Single,Student,CAR - Cordillera Administrative Region,1,Online Stores,1,0,1,0,0,0,0,0,1,0,0,0,0
2,20,Male,Single,Student,Region III - Central Luzon,2,Online Stores,1,1,1,1,1,0,0,0,1,1,0,0,0
3,21,Female,Single,Student,Region IV-A - CALABARZON,4,Online Stores,1,1,1,1,1,0,0,1,0,0,0,0,0
4,22,Female,Single,Student,Region IV-A - CALABARZON,5,Online Stores,1,1,1,1,0,1,0,0,1,0,0,1,1


In [61]:
# Get rules from all male baskets

for i in baskets.index:
    # Indicate which basket to show rules:
    row = baskets.iloc[i]
    basket_name = row["name"]
    basket = row["basket"]
    min_support = row["min_support"]
    print(f'basket name: {basket_name}')

    # Sort the holiday basket
    # Sort the basket before dropping user_data_cols
    basket_sorted = basket[basket["Gender"] == "Male"].drop(user_data_cols, axis=1)
    
    # Get frequent item set
    freq_items = apriori(basket_sorted, min_support = min_support, use_colnames=True)
    freq_items.sort_values("support", ascending=False)

    print(f'length of frequent item sets: {len(freq_items)}')
    print(f'minimum support: {min_support}')
    # Get rules 
    rules = association_rules(freq_items, metric="lift", min_threshold=1)
    display(rules.sort_values("support", ascending=False).head(10))

basket name: holidays_basket
length of frequent item sets: 74
minimum support: 0.05


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(Apparel/Clothing),(Sports & Travel),0.326531,0.204082,0.142857,0.437500,2.143750,0.076218,1.414966
9,(Sports & Travel),(Apparel/Clothing),0.204082,0.326531,0.142857,0.700000,2.143750,0.076218,2.244898
25,(Tech Accessories),(Home Entertainment),0.183673,0.244898,0.122449,0.666667,2.722222,0.077468,2.265306
16,(Apparel/Clothing),(Tech Accessories),0.326531,0.183673,0.122449,0.375000,2.041667,0.062474,1.306122
2,(Apparel/Clothing),"(Toys, Games, Collectibles)",0.326531,0.265306,0.122449,0.375000,1.413462,0.035818,1.175510
3,"(Toys, Games, Collectibles)",(Apparel/Clothing),0.265306,0.326531,0.122449,0.461538,1.413462,0.035818,1.250729
24,(Home Entertainment),(Tech Accessories),0.244898,0.183673,0.122449,0.500000,2.722222,0.077468,1.632653
17,(Tech Accessories),(Apparel/Clothing),0.183673,0.326531,0.122449,0.666667,2.041667,0.062474,2.020408
41,(Tech Accessories),"(Toys, Games, Collectibles)",0.183673,0.265306,0.102041,0.555556,2.094017,0.053311,1.653061
40,"(Toys, Games, Collectibles)",(Tech Accessories),0.265306,0.183673,0.102041,0.384615,2.094017,0.053311,1.326531


basket name: bdays_basket
length of frequent item sets: 71
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
38,"(Toys, Games, Collectibles)",(Tech Accessories),0.469388,0.346939,0.244898,0.521739,1.503836,0.082049,1.365492
39,(Tech Accessories),"(Toys, Games, Collectibles)",0.346939,0.469388,0.244898,0.705882,1.503836,0.082049,1.804082
33,"(Toys, Games, Collectibles)",(Wearable Accessories),0.469388,0.346939,0.244898,0.521739,1.503836,0.082049,1.365492
32,(Wearable Accessories),"(Toys, Games, Collectibles)",0.346939,0.469388,0.244898,0.705882,1.503836,0.082049,1.804082
0,(Apparel/Clothing),"(Toys, Games, Collectibles)",0.306122,0.469388,0.204082,0.666667,1.420290,0.060392,1.591837
6,(Apparel/Clothing),(Wearable Accessories),0.306122,0.346939,0.204082,0.666667,1.921569,0.097876,1.959184
7,(Wearable Accessories),(Apparel/Clothing),0.346939,0.306122,0.204082,0.588235,1.921569,0.097876,1.685131
1,"(Toys, Games, Collectibles)",(Apparel/Clothing),0.469388,0.306122,0.204082,0.434783,1.420290,0.060392,1.227630
60,(Laptops/Computers),(Wearable Accessories),0.285714,0.346939,0.183673,0.642857,1.852941,0.084548,1.828571
61,(Wearable Accessories),(Laptops/Computers),0.346939,0.285714,0.183673,0.529412,1.852941,0.084548,1.517857


basket name: periods_basket
length of frequent item sets: 72
minimum support: 0.05


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(Laptops/Computers),(Health & Personal Care),0.775510,0.714286,0.612245,0.789474,1.105263,0.058309,1.357143
9,(Health & Personal Care),(Laptops/Computers),0.714286,0.775510,0.612245,0.857143,1.105263,0.058309,1.571429
37,(Tech Accessories),(Laptops/Computers),0.714286,0.775510,0.612245,0.857143,1.105263,0.058309,1.571429
36,(Laptops/Computers),(Tech Accessories),0.775510,0.714286,0.612245,0.789474,1.105263,0.058309,1.357143
21,(Tech Accessories),(Bags),0.714286,0.714286,0.591837,0.828571,1.160000,0.081633,1.666667
20,(Bags),(Tech Accessories),0.714286,0.714286,0.591837,0.828571,1.160000,0.081633,1.666667
19,(Bags),(Laptops/Computers),0.714286,0.775510,0.591837,0.828571,1.068421,0.037901,1.309524
18,(Laptops/Computers),(Bags),0.775510,0.714286,0.591837,0.763158,1.068421,0.037901,1.206349
14,(Stationery),(Bags),0.673469,0.714286,0.551020,0.818182,1.145455,0.069971,1.571429
15,(Bags),(Stationery),0.714286,0.673469,0.551020,0.771429,1.145455,0.069971,1.428571


basket name: summer_basket
length of frequent item sets: 88
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(Clothing),(Water Bottles),0.734694,0.551020,0.408163,0.555556,1.008230,0.003332,1.010204
7,(Water Bottles),(Clothing),0.551020,0.734694,0.408163,0.740741,1.008230,0.003332,1.023324
28,(Water Bottles),(Sunscreen/Heat-Protectant Skin Products),0.551020,0.428571,0.367347,0.666667,1.555556,0.131195,1.714286
29,(Sunscreen/Heat-Protectant Skin Products),(Water Bottles),0.428571,0.551020,0.367347,0.857143,1.555556,0.131195,3.142857
2,(Clothing),(Towels),0.734694,0.408163,0.346939,0.472222,1.156944,0.047064,1.121375
3,(Towels),(Clothing),0.408163,0.734694,0.346939,0.850000,1.156944,0.047064,1.768707
36,(Towels),(Water Bottles),0.408163,0.551020,0.326531,0.800000,1.451852,0.101624,2.244898
37,(Water Bottles),(Towels),0.551020,0.408163,0.326531,0.592593,1.451852,0.101624,1.452690
107,(Clothing),"(Towels, Water Bottles)",0.734694,0.326531,0.285714,0.388889,1.190972,0.045814,1.102041
104,"(Clothing, Water Bottles)",(Towels),0.408163,0.408163,0.285714,0.700000,1.715000,0.119117,1.972789


basket name: bts_basket
length of frequent item sets: 257
minimum support: 0.2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Pen/Marker/Pencil/Scriber),(Notebooks),0.938776,0.857143,0.816327,0.869565,1.014493,0.011662,1.095238
3,(Notebooks),(Pen/Marker/Pencil/Scriber),0.857143,0.938776,0.816327,0.952381,1.014493,0.011662,1.285714
0,(Pen/Marker/Pencil/Scriber),(Paper/Envelopes/Post-it Notes/),0.938776,0.734694,0.714286,0.760870,1.035628,0.024573,1.109462
1,(Paper/Envelopes/Post-it Notes/),(Pen/Marker/Pencil/Scriber),0.734694,0.938776,0.714286,0.972222,1.035628,0.024573,2.204082
10,(Notebooks),(Paper/Envelopes/Post-it Notes/),0.857143,0.734694,0.632653,0.738095,1.004630,0.002915,1.012987
11,(Paper/Envelopes/Post-it Notes/),(Notebooks),0.734694,0.857143,0.632653,0.861111,1.004630,0.002915,1.028571
87,(Paper/Envelopes/Post-it Notes/),"(Pen/Marker/Pencil/Scriber, Notebooks)",0.734694,0.816327,0.612245,0.833333,1.020833,0.012495,1.102041
82,"(Pen/Marker/Pencil/Scriber, Notebooks)",(Paper/Envelopes/Post-it Notes/),0.816327,0.734694,0.612245,0.750000,1.020833,0.012495,1.061224
83,"(Pen/Marker/Pencil/Scriber, Paper/Envelopes/Po...",(Notebooks),0.714286,0.857143,0.612245,0.857143,1.000000,0.000000,1.000000
84,"(Notebooks, Paper/Envelopes/Post-it Notes/)",(Pen/Marker/Pencil/Scriber),0.632653,0.938776,0.612245,0.967742,1.030856,0.018326,1.897959


In [62]:
# Get rules from all male baskets

for i in baskets.index:
    # Indicate which basket to show rules:
    row = baskets.iloc[i]
    basket_name = row["name"]
    basket = row["basket"]
    min_support = row["min_support"]
    print(f'basket name: {basket_name}')

    # Sort the holiday basket
    # Sort the basket before dropping user_data_cols
    basket_sorted = basket[basket["Gender"] == "Female"].drop(user_data_cols, axis=1)
    
    # Get frequent item set
    freq_items = apriori(basket_sorted, min_support = min_support, use_colnames=True)
    freq_items.sort_values("support", ascending=False)

    print(f'length of frequent item sets: {len(freq_items)}')
    print(f'minimum support: {min_support}')
    # Get rules 
    rules = association_rules(freq_items, metric="lift", min_threshold=1)
    display(rules.sort_values("support", ascending=False).head(10))

basket name: holidays_basket
length of frequent item sets: 78
minimum support: 0.05


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Wearable Accessories),(Apparel/Clothing),0.225,0.325,0.150,0.666667,2.051282,0.076875,2.025000
14,(Apparel/Clothing),(Wearable Accessories),0.325,0.225,0.150,0.461538,2.051282,0.076875,1.439286
119,(Bags),"(Apparel/Clothing, Wearable Accessories)",0.175,0.150,0.125,0.714286,4.761905,0.098750,2.975000
118,(Wearable Accessories),"(Apparel/Clothing, Bags)",0.225,0.125,0.125,0.555556,4.444444,0.096875,1.968750
117,(Apparel/Clothing),"(Wearable Accessories, Bags)",0.325,0.125,0.125,0.384615,3.076923,0.084375,1.421875
116,"(Wearable Accessories, Bags)",(Apparel/Clothing),0.125,0.325,0.125,1.000000,3.076923,0.084375,inf
115,"(Apparel/Clothing, Bags)",(Wearable Accessories),0.125,0.225,0.125,1.000000,4.444444,0.096875,inf
114,"(Apparel/Clothing, Wearable Accessories)",(Bags),0.150,0.175,0.125,0.833333,4.761905,0.098750,4.950000
8,(Apparel/Clothing),(Bags),0.325,0.175,0.125,0.384615,2.197802,0.068125,1.340625
9,(Bags),(Apparel/Clothing),0.175,0.325,0.125,0.714286,2.197802,0.068125,2.362500


basket name: bdays_basket
length of frequent item sets: 32
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(Wearable Accessories),(Apparel/Clothing),0.35,0.35,0.25,0.714286,2.040816,0.1275,2.275000
4,(Apparel/Clothing),(Wearable Accessories),0.35,0.35,0.25,0.714286,2.040816,0.1275,2.275000
0,(Apparel/Clothing),(Bags),0.35,0.30,0.20,0.571429,1.904762,0.0950,1.633333
1,(Bags),(Apparel/Clothing),0.30,0.35,0.20,0.666667,1.904762,0.0950,1.950000
19,(Bags),(Wearable Accessories),0.30,0.35,0.20,0.666667,1.904762,0.0950,1.950000
18,(Wearable Accessories),(Bags),0.35,0.30,0.20,0.571429,1.904762,0.0950,1.633333
35,(Bags),"(Apparel/Clothing, Wearable Accessories)",0.30,0.25,0.15,0.500000,2.000000,0.0750,1.500000
32,"(Wearable Accessories, Bags)",(Apparel/Clothing),0.20,0.35,0.15,0.750000,2.142857,0.0800,2.600000
2,(Apparel/Clothing),(Books),0.35,0.25,0.15,0.428571,1.714286,0.0625,1.312500
3,(Books),(Apparel/Clothing),0.25,0.35,0.15,0.600000,1.714286,0.0625,1.625000


basket name: periods_basket
length of frequent item sets: 42
minimum support: 0.05


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
41,(Sports & Travel),(Laptops/Computers),0.700,0.700,0.650,0.928571,1.326531,0.1600,4.200000
40,(Laptops/Computers),(Sports & Travel),0.700,0.700,0.650,0.928571,1.326531,0.1600,4.200000
59,(Cameras/Camera Accessories),(Laptops/Computers),0.675,0.700,0.625,0.925926,1.322751,0.1525,4.050000
58,(Laptops/Computers),(Cameras/Camera Accessories),0.700,0.675,0.625,0.892857,1.322751,0.1525,3.033333
23,(Stationery),(Laptops/Computers),0.700,0.700,0.600,0.857143,1.224490,0.1100,2.100000
22,(Laptops/Computers),(Stationery),0.700,0.700,0.600,0.857143,1.224490,0.1100,2.100000
17,(Sports & Travel),(Stationery),0.700,0.700,0.600,0.857143,1.224490,0.1100,2.100000
16,(Stationery),(Sports & Travel),0.700,0.700,0.600,0.857143,1.224490,0.1100,2.100000
50,(Laptops/Computers),(Automobile Accessories),0.700,0.675,0.600,0.857143,1.269841,0.1275,2.275000
51,(Automobile Accessories),(Laptops/Computers),0.675,0.700,0.600,0.888889,1.269841,0.1275,2.700000


basket name: summer_basket
length of frequent item sets: 149
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Clothing),(Sunscreen/Heat-Protectant Skin Products),0.725,0.725,0.575,0.793103,1.093936,0.049375,1.329167
1,(Sunscreen/Heat-Protectant Skin Products),(Clothing),0.725,0.725,0.575,0.793103,1.093936,0.049375,1.329167
10,(Umbrellas),(Sunscreen/Heat-Protectant Skin Products),0.600,0.725,0.475,0.791667,1.091954,0.040000,1.320000
11,(Sunscreen/Heat-Protectant Skin Products),(Umbrellas),0.725,0.600,0.475,0.655172,1.091954,0.040000,1.160000
28,(Water Bottles),(Sunscreen/Heat-Protectant Skin Products),0.550,0.725,0.450,0.818182,1.128527,0.051250,1.512500
29,(Sunscreen/Heat-Protectant Skin Products),(Water Bottles),0.725,0.550,0.450,0.620690,1.128527,0.051250,1.186364
8,(Clothing),(Water Bottles),0.725,0.550,0.400,0.551724,1.003135,0.001250,1.003846
9,(Water Bottles),(Clothing),0.550,0.725,0.400,0.727273,1.003135,0.001250,1.008333
17,(Umbrellas),(Water Bottles),0.600,0.550,0.375,0.625000,1.136364,0.045000,1.200000
16,(Water Bottles),(Umbrellas),0.550,0.600,0.375,0.681818,1.136364,0.045000,1.257143


basket name: bts_basket
length of frequent item sets: 478
minimum support: 0.2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,(Paper/Envelopes/Post-it Notes/),(Umbrella),0.900,0.675,0.625,0.694444,1.028807,0.017500,1.063636
20,(Umbrella),(Paper/Envelopes/Post-it Notes/),0.675,0.900,0.625,0.925926,1.028807,0.017500,1.350000
34,(Umbrella),(Notebooks),0.675,0.925,0.625,0.925926,1.001001,0.000625,1.012500
35,(Notebooks),(Umbrella),0.925,0.675,0.625,0.675676,1.001001,0.000625,1.002083
11,(Bags/School Bags),(Pen/Marker/Pencil/Scriber),0.625,0.975,0.625,1.000000,1.025641,0.015625,inf
10,(Pen/Marker/Pencil/Scriber),(Bags/School Bags),0.975,0.625,0.625,0.641026,1.025641,0.015625,1.044643
36,(Notebooks),(Bags/School Bags),0.925,0.625,0.600,0.648649,1.037838,0.021875,1.067308
37,(Bags/School Bags),(Notebooks),0.625,0.925,0.600,0.960000,1.037838,0.021875,1.875000
169,(Bags/School Bags),"(Pen/Marker/Pencil/Scriber, Notebooks)",0.625,0.900,0.600,0.960000,1.066667,0.037500,2.500000
122,"(Umbrella, Pen/Marker/Pencil/Scriber)",(Paper/Envelopes/Post-it Notes/),0.650,0.900,0.600,0.923077,1.025641,0.015000,1.300000


In [63]:
basket.columns

Index(['Age', 'Gender', 'Marital Status', 'Employment Status',
       'Current Region', 'Frequency', 'Where', 'Pen/Marker/Pencil/Scriber',
       'Paper/Envelopes/Post-it Notes/', 'Notebooks', 'Academic Books',
       'Art Materials', 'Laptop', 'Phone/Tablet',
       'Tech Products/Tech Accessories', 'School Uniforms/Clothing',
       'Umbrella', 'Raincoat', 'Bags/School Bags', 'Make up/Beauty Products'],
      dtype='object')

In [65]:
# Get rules from all male baskets

for i in baskets.index:
    # Indicate which basket to show rules:
    row = baskets.iloc[i]
    basket_name = row["name"]
    basket = row["basket"]
    min_support = row["min_support"]
    print(f'basket name: {basket_name}')
    # Sort the holiday basket
    # Sort the basket before dropping user_data_cols
    basket_sorted = basket[basket["Current Region"] == "Region I - Ilocos Region"].drop(user_data_cols, axis=1)
    
    # Get frequent item set
    freq_items = apriori(basket_sorted, min_support = min_support, use_colnames=True)
    freq_items.sort_values("support", ascending=False)

    print(f'length of frequent item sets: {len(freq_items)}')
    print(f'minimum support: {min_support}')
    # Get rules 
    rules = association_rules(freq_items, metric="lift", min_threshold=1)
    display(rules.sort_values("support", ascending=False).head(10))

basket name: holidays_basket
length of frequent item sets: 101
minimum support: 0.05


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
24,(Food/Groceries),(Sports & Travel),0.142857,0.200000,0.142857,1.000000,5.000000,0.114286,inf
25,(Sports & Travel),(Food/Groceries),0.200000,0.142857,0.142857,0.714286,5.000000,0.114286,3.000000
8,(Apparel/Clothing),(Sports & Travel),0.285714,0.200000,0.114286,0.400000,2.000000,0.057143,1.333333
9,(Sports & Travel),(Apparel/Clothing),0.200000,0.285714,0.114286,0.571429,2.000000,0.057143,1.666667
56,(Wearable Accessories),(Sports & Travel),0.257143,0.200000,0.114286,0.444444,2.222222,0.062857,1.440000
57,(Sports & Travel),(Wearable Accessories),0.200000,0.257143,0.114286,0.571429,2.222222,0.062857,1.733333
76,"(Food/Groceries, Apparel/Clothing)",(Sports & Travel),0.085714,0.200000,0.085714,1.000000,5.000000,0.068571,inf
77,"(Food/Groceries, Sports & Travel)",(Apparel/Clothing),0.142857,0.285714,0.085714,0.600000,2.100000,0.044898,1.785714
78,"(Apparel/Clothing, Sports & Travel)",(Food/Groceries),0.114286,0.142857,0.085714,0.750000,5.250000,0.069388,3.428571
0,(Food/Groceries),(Apparel/Clothing),0.142857,0.285714,0.085714,0.600000,2.100000,0.044898,1.785714


basket name: bdays_basket
length of frequent item sets: 47
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11,(Wearable Accessories),(Apparel/Clothing),0.428571,0.314286,0.285714,0.666667,2.121212,0.151020,2.057143
10,(Apparel/Clothing),(Wearable Accessories),0.314286,0.428571,0.285714,0.909091,2.121212,0.151020,6.285714
22,(Wearable Accessories),"(Toys, Games, Collectibles)",0.428571,0.371429,0.228571,0.533333,1.435897,0.069388,1.346939
23,"(Toys, Games, Collectibles)",(Wearable Accessories),0.371429,0.428571,0.228571,0.615385,1.435897,0.069388,1.485714
26,"(Toys, Games, Collectibles)",(Tech Accessories),0.371429,0.228571,0.200000,0.538462,2.355769,0.115102,1.671429
27,(Tech Accessories),"(Toys, Games, Collectibles)",0.228571,0.371429,0.200000,0.875000,2.355769,0.115102,5.028571
72,"(Apparel/Clothing, Wearable Accessories)",(Books),0.285714,0.200000,0.142857,0.500000,2.500000,0.085714,1.600000
24,(Laptops/Computers),"(Toys, Games, Collectibles)",0.228571,0.371429,0.142857,0.625000,1.682692,0.057959,1.676190
25,"(Toys, Games, Collectibles)",(Laptops/Computers),0.371429,0.228571,0.142857,0.384615,1.682692,0.057959,1.253571
28,(Wearable Accessories),(Bags),0.428571,0.200000,0.142857,0.333333,1.666667,0.057143,1.200000


basket name: periods_basket
length of frequent item sets: 73
minimum support: 0.05


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
68,(Laptops/Computers),(Tech Accessories),0.828571,0.771429,0.714286,0.862069,1.117497,0.075102,1.657143
69,(Tech Accessories),(Laptops/Computers),0.771429,0.828571,0.714286,0.925926,1.117497,0.075102,2.314286
77,(Power Tools),(Tech Accessories),0.742857,0.771429,0.685714,0.923077,1.196581,0.112653,2.971429
76,(Tech Accessories),(Power Tools),0.771429,0.742857,0.685714,0.888889,1.196581,0.112653,2.314286
73,(Power Tools),(Laptops/Computers),0.742857,0.828571,0.685714,0.923077,1.114058,0.070204,2.228571
72,(Laptops/Computers),(Power Tools),0.828571,0.742857,0.685714,0.827586,1.114058,0.070204,1.491429
276,(Tech Accessories),"(Laptops/Computers, Power Tools)",0.771429,0.685714,0.657143,0.851852,1.242284,0.128163,2.121429
60,(Laptops/Computers),(Cameras/Camera Accessories),0.828571,0.714286,0.657143,0.793103,1.110345,0.065306,1.380952
61,(Cameras/Camera Accessories),(Laptops/Computers),0.714286,0.828571,0.657143,0.920000,1.110345,0.065306,2.142857
62,(Cameras/Camera Accessories),(Tech Accessories),0.714286,0.771429,0.657143,0.920000,1.192593,0.106122,2.857143


basket name: summer_basket
length of frequent item sets: 215
minimum support: 0.1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(Clothing),(Water Bottles),0.771429,0.714286,0.571429,0.740741,1.037037,0.020408,1.102041
7,(Water Bottles),(Clothing),0.714286,0.771429,0.571429,0.800000,1.037037,0.020408,1.142857
43,(Water Bottles),(Towels),0.714286,0.571429,0.514286,0.720000,1.260000,0.106122,1.530612
42,(Towels),(Water Bottles),0.571429,0.714286,0.514286,0.900000,1.260000,0.106122,2.857143
31,(Sunscreen/Heat-Protectant Skin Products),(Water Bottles),0.600000,0.714286,0.514286,0.857143,1.200000,0.085714,2.000000
30,(Water Bottles),(Sunscreen/Heat-Protectant Skin Products),0.714286,0.600000,0.514286,0.720000,1.200000,0.085714,1.428571
0,(Clothing),(Sunscreen/Heat-Protectant Skin Products),0.771429,0.600000,0.485714,0.629630,1.049383,0.022857,1.080000
1,(Sunscreen/Heat-Protectant Skin Products),(Clothing),0.600000,0.771429,0.485714,0.809524,1.049383,0.022857,1.200000
26,(Towels),(Sunscreen/Heat-Protectant Skin Products),0.571429,0.600000,0.428571,0.750000,1.250000,0.085714,1.600000
27,(Sunscreen/Heat-Protectant Skin Products),(Towels),0.600000,0.571429,0.428571,0.714286,1.250000,0.085714,1.500000


basket name: bts_basket
length of frequent item sets: 833
minimum support: 0.2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Pen/Marker/Pencil/Scriber),(Paper/Envelopes/Post-it Notes/),0.942857,0.828571,0.800000,0.848485,1.024033,0.018776,1.131429
1,(Paper/Envelopes/Post-it Notes/),(Pen/Marker/Pencil/Scriber),0.828571,0.942857,0.800000,0.965517,1.024033,0.018776,1.657143
28,(Umbrella),(Notebooks),0.714286,0.914286,0.714286,1.000000,1.093750,0.061224,inf
82,"(Pen/Marker/Pencil/Scriber, Notebooks)",(Paper/Envelopes/Post-it Notes/),0.857143,0.828571,0.714286,0.833333,1.005747,0.004082,1.028571
83,"(Notebooks, Paper/Envelopes/Post-it Notes/)",(Pen/Marker/Pencil/Scriber),0.742857,0.942857,0.714286,0.961538,1.019814,0.013878,1.485714
84,(Pen/Marker/Pencil/Scriber),"(Notebooks, Paper/Envelopes/Post-it Notes/)",0.942857,0.742857,0.714286,0.757576,1.019814,0.013878,1.060714
85,(Paper/Envelopes/Post-it Notes/),"(Pen/Marker/Pencil/Scriber, Notebooks)",0.828571,0.857143,0.714286,0.862069,1.005747,0.004082,1.035714
29,(Notebooks),(Umbrella),0.914286,0.714286,0.714286,0.781250,1.093750,0.061224,1.306122
30,(Notebooks),(Bags/School Bags),0.914286,0.714286,0.685714,0.750000,1.050000,0.032653,1.142857
31,(Bags/School Bags),(Notebooks),0.714286,0.914286,0.685714,0.960000,1.050000,0.032653,2.142857
